In [1]:
import os
import requests
from bs4 import BeautifulSoup

In [2]:
session = requests.Session()
login_url = 'https://onlinejudge.org/index.php?option=com_comprofiler&task=login'
response = session.get(login_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
form = soup.find('form', id='mod_loginform')
hidden_inputs = form.find_all('input', type='hidden')
form_data = {input.get('name'): input.get('value') for input in hidden_inputs}

In [4]:
form_data['username'] = 'admin'
form_data['passwd'] = 'admin'
form_data['remember'] = 'yes'
form_data['Submit'] = 'Login'

In [5]:
login_response = session.post(form['action'], data=form_data)
assert 'Logout' in login_response.text, 'Login failed!'

In [6]:
category_url = 'https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&category=606'
response = session.get(category_url)
soup = BeautifulSoup(response.text, 'html.parser')

categories = {
    'Super Easy': '607',
    'Easy': '608',
    'Medium': '609'
}

category_links = {name: f"https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&category={id}" for name, id in categories.items()}


In [ ]:
problem_data = {}

for category, link in category_links.items():
    response = session.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    problems = soup.find_all('a', href=True)
    
    print(category)
    problem_data[category] = []
    for a in problems:
        if 'problem' in a['href']:
            problem_url = f"https://onlinejudge.org{a['href']}"
            problem_name_full = a.text.strip()
            problem_name = problem_name_full.split(' - ')[1]
            problem_id = problem_url.split('problem=')[1]
            problem_data[category].append((problem_name, problem_url, problem_id))
            print(problem_name)


In [ ]:
base_folder = 'Getting Started: The Easy Problems'

for category, problems in problem_data.items():
    category_folder = os.path.join(base_folder, category)
    for problem_name, problem_url, problem_id in problems:
        problem_folder = os.path.join(category_folder, problem_name)
        
        if not os.path.exists(problem_folder):
            continue

        # Find the .cpp file in the problem's folder
        solution_file = None
        for file in os.listdir(problem_folder):
            if file.endswith('.cpp'):
                solution_file = os.path.join(problem_folder, file)
                break
        
        if solution_file and os.path.exists(solution_file):
            # Read the solution file
            with open(solution_file, 'r') as file:
                solution_code = file.read()
                
            # Navigate to the problem submission page
            submission_url = f"https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&page=submit_problem&problemid={problem_id}&category=0"
            submission_response = session.get(submission_url)
            submission_soup = BeautifulSoup(submission_response.text, 'html.parser')
            
            # Find the submission form and extract necessary fields
            form = submission_soup.find('form', {'action': 'index.php?option=com_onlinejudge&Itemid=8&page=save_submission'})
            form_data = {input.get('name'): input.get('value') for input in form.find_all('input', type='hidden')}
            
            form_data['language'] = '5'
            form_data['code'] = solution_code
            
            # Submit the solution
            submit_url = 'https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&page=save_submission'
            submission_response = session.post(submit_url, data=form_data)
            
            print(f"Successfully submitted solution for problem {problem_name}")
